## N-gram Language Models

In [8]:
import nltk
nltk.download('punkt')

nltk_data_path = "Data/punkt.zip"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qfu88\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


### Load the dataset

Create a function to load the data, This function should accomplish the following:

Extract sentences from the data file. Of course, depending on the nature of the task at hand, what constitutes a sentence can vary. In the context of this assignment, we will define a sentence as just a line of a sonnet, regardless of the punctuation at end. In addition, we will ignore the boundaries of the sonnets --- that is, we are not dealing with  154
  individual sonnets but rather  154×14=2156
  sentences (actually only  2155
  sentences, as Sonnet 99 has 15 lines but Sonnet 126 has only 12). We encourage you to explore alternative definitions of a sentence on your own; for example, an entire sonnet could be modelled as a sentence. Finally, make sure that the newline character \n at the end of each line is dropped.
  
  
Tokenise each extracted sentence. While it's ambiguous what a sentence is, what constitutes a "word" is even more task-dependent. Do punctuations count as "words"? Are two "words" with the same spelling but different casing considered identical? Since what a text file contains is nothing more than a squence of characters, there are many possible ways of grouping these characters to form "words" that are subsequently taken as input by a program. To distinguish what's actually taken as input by a program from a linguistic word, we call the former a token. The process of producing a list of tokens out of a sentence is then called tokenisation. At this step, you will first lower-case each sentence extracted from the previous step entirely and then tokenise each lower-cased sentence. You may use any tokeniser of your choice, such as word_tokenize from the nltk library. The grading of the assignment doesn't depend on your choice of the tokeniser.

In [9]:
from nltk.tokenize import word_tokenize


def load_data():
    """
    Load text data from a file and produce a list of token lists
    """
    
    sentences = []
    
    with open("Data\THE_SONNETS.txt", "r") as file:
        lines = file.readlines()
        
        for line in lines:
            # Remove the newline character at the end
            line = line.strip()
            # Skip empty lines or lines that are just numbers (indicating sonnet numbers)
            if not line or line.isdigit():
                continue
            
            line = line.lower()
            # Tokenize the sentence
            tokens = word_tokenize(line)
            
            sentences.append(tokens)

    
    return sentences

### Build vocabulary

Next, we need a "vocabulary" that contains all the unique tokens. Moreover, as mentioned in the lecture, we often pad a sentence with <s> and </s> to indicate its start and end when working with n-gram language models; therefore, these two special tokens should also be included in our vocabulary. 

In [13]:
def build_vocab(sentences):
    """
    Take a list of sentences and return a vocab
    """
    
    vocab = set()
    for sentence in sentences:
        for token in sentence:
            vocab.add(token)
    vocab.add("<s>")
    vocab.add("</s>")
    
    vocab = list(vocab)
    
    return vocab

###  Generate all  𝑛-grams
 

Now let's write a function to generate all  𝑛
 -grams for each sentence. This can be accomplished in two steps:

Pad each sentence with <s> and </s> for  𝑛≥2
 . You need  𝑛−1
  paddings on both ends of a sentence, so that there are two  𝑛
 -grams that model the first and the last token, respectively. You may implement the padding function yourself or use the pad_both_ends function from the nltk library.
Generate  𝑛
 -grams on the padded sentences. Check out the ngrams function from nltk. For a sentence with  ℓ
  tokens excluding paddings, the maximum number of possible  𝑛
 -grams generated from its padded version should be  ℓ+𝑛−1
 . Think about why.

In [16]:
from nltk.util import ngrams


def pad_sentence(sentence, n):
    
    return ['<s>'] * (n-1) + sentence + ['</s>'] * (n-1)

def build_ngrams(n, sentences):
    """
    Take a list of unpadded sentences and create all n-grams as specified by the argument "n" for each sentence
    """
    
    all_ngrams = []
    
    for sentence in sentences:
        padded_sentence = list(pad_sentence(sentence, n))
        
        all_ngrams.append(list(ngrams(padded_sentence, n)))
       
    
    return all_ngrams

### Guess the next token

Assume we are now working with bi-grams. What is the most likely token that comes after the sequence "s s s", and how likely? Remember that a bi-gram language model is essentially a first-order Markov Chain. So, what determines the next state in a first-order Markov Chain?

In [21]:
from collections import Counter, defaultdict

def bigram_next_token(start_tokens=("<s>", ) * 3):
    """
    Take some starting tokens and produce the most likely token that follows under a bi-gram model
    """
    
    next_token, prob = None, None
    #load_data() in order to have sentences to work with 
    #create ngrams using build_ngrams() function 
    #create empty list for tokens 

    #iterate through the ngrams created from the build_ngrams() function 
        #iterate through the bigrams in ngrams 
            #if bigrams index 0 is equal to last start_tokens
                #append the last bigram to empty list created for tokens 

    #HINT: use .most_common from Counter of tokens to identify what the next token would be 
    #calculate the probability of the token 
  
    #load data and create bi-grams
    sentences = load_data()
    bi_grams = build_ngrams(2, sentences)
    
    
    #Flatten the list of lists to have a flat list of bi-grams
    flat_bigrams = [bigram for sublist in bi_grams for bigram in sublist]
    
    #Create a Counter to count occurrences of each bi-gram
    bigram_counter = Counter(flat_bigrams)
    
    # Extract the condition token, i.e., the token we want to compute the conditional probability on
    condition_token = start_tokens[-1]
    
    # Create a dictionary to store conditional probabilities
    cond_prob = {}
    for bigram, count in bigram_counter.items():
        if bigram[0] == condition_token:
            cond_prob[bigram[1]] = count / sum([val for key, val in bigram_counter.items() if key[0] == condition_token])
    
    # Step 5: Identify the most likely next token and its probability
    next_token, prob = max(cond_prob.items(), key=lambda x: x[1])
    

    
    #raise NotImplementedError()
    
    return next_token, prob

### Train an  𝑛-gram language model

Now we are well positioned to start training an  𝑛
 -gram language model. We can fit a language model using the MLE class from nltk.lm. It requires two inputs: a list of all  𝑛
 -grams for each sentence and a vocabulary, both of which you have already written a function to build. Now it's time to put them together to work.

In [24]:
from nltk.lm import MLE

def train_ngram_lm(n):
    """
    Train a n-gram language model as specified by the argument "n"
    """
    
    lm = MLE(n)
    
    # Load the sentences
    sentences = load_data()
    
    # Build n-grams, get a flat list of n-grams
    all_ngrams = [gram for sent in build_ngrams(n, sentences) for gram in sent]
    
    # Build vocabulary
    vocab = build_vocab(sentences)
    
    # Fit the model with n-grams
    lm.fit([all_ngrams], vocabulary_text=vocab)
    
    return lm


FINALLY, are you ready to compose sonnets like the real Shakespeare?! We provide some starter code below, but absolutely feel free to modify any parts of it on your own. It'd be interesting to see how the "authenticity" of the sonnets is related to the parameter  𝑛
 . Do the sonnets feel more Shakespeare when you increase  𝑛
 ?

In [28]:
# Every time it runs, depending on how drunk it is, a different sonnet is written. 
n = 3
num_lines = 14
num_words_per_line = 8
text_seed = ["<s>"] * (n - 1)

lm = train_ngram_lm(n)

sonnet = []
while len(sonnet) < num_lines:
    while True:  # keep generating a line until success
        try:
            line = lm.generate(num_words_per_line, text_seed=text_seed)
        except ValueError:  # the generation is not always successful. need to capture exceptions
            continue
        else:
            line = [x for x in line if x not in ["<s>", "</s>"]]
            sonnet.append(" ".join(line))
            break

# pretty-print your sonnet
print("\n".join(sonnet))

be as thy sweet self prove .
even so doth she abuse me ,
within thine own deep sunken eyes , her
on your broad main doth wilfully appear .
hath motion , and mine only care ,
straight in her heart did mercy come ,
and often is his due ,
whether we are mended , or more
he lends thee virtue , and therefore have
no longer mourn for me than spurring to
why with the time do i find ,
when most i wink then do thy office
and him as fist doth bind .
strikes each in each by mutual ordering ;


### Hidden Markov Models (HMMs)